In [1]:
from faker import Faker
import google.auth
from google.cloud.sql.connector import Connector
from google.auth.transport.requests import Request
import sqlalchemy
import random

In [2]:
# initialize parameters
# INSTANCE_CONNECTION_NAME = f"{project_id}:{region}:{instance_name}" # i.e demo-project:us-central1:demo-instance
# print(f"Your instance connection name is: {INSTANCE_CONNECTION_NAME}")
DB_USER = "rustom"
DB_PASS = "hasbulla"
DB_NAME = "family-hub-database"

INSTANCE_CONNECTION_NAME = 'familyhub-366019:us-central1:family-hub-dev'
current_user = !gcloud auth list --filter=status:ACTIVE --format="value(account)"

# initialize Connector object
connector = Connector()

# function to return the database connection object
def getconn():
    conn = connector.connect(
        INSTANCE_CONNECTION_NAME,
        "pymysql",
        user=DB_USER,
        password=DB_PASS,
        db=DB_NAME
    )
    return conn

# create connection pool with 'creator' argument to our connection object function
pool = sqlalchemy.create_engine(
    "mysql+pymysql://",
    creator=getconn,
)

IAM_USER = current_user[0].split("@")[0]

# get application default credentials of IAM user (current logged in user)
credentials, project = google.auth.default()

# refresh credentials if expired
if not credentials.valid:
      request = Request()
      credentials.refresh(request)


In [3]:
with pool.connect() as db_conn:
    results = db_conn.execute('show tables;').fetchall()
    
    for row in results: 
        print(row)
    

('Family',)
('Membership',)
('SubscriptionService',)
('University',)
('User',)


In [26]:
from unicodedata import unidata_version

with pool.connect() as con: 
    insert_statement = sqlalchemy.text(
        'insert into University (universityID, universityName, city) values (:universityID, :universityName, :city)'
    )

    con.execute(insert_statement, 
        universityID=0, 
        universityName='University of Illinois at Urbana-Champaign', 
        city='Champaign, IL'
        )
    
    con.execute(insert_statement, 
        universityID=1, 
        universityName='Harvard University',
        city='Cambridge, MA'
        )

    con.execute(insert_statement, 
        universityID=2,
        universityName='Harper College',
        city='Palatine, IL'
        )

    con.execute(insert_statement, 
        universityID=3, 
        universityName='Northwestern University', 
        city='Evanston, IL'
        )

    con.execute(insert_statement, 
        universityID=4, 
        universityName='Massachusetts Institute of Technology', 
        city='Cambridge, MA'
        )

    con.execute(insert_statement, 
        universityID=5, 
        universityName='Yale University',
        city='New Haven, CT'
        )

In [60]:
with pool.connect() as con: 
    results = con.execute('select * from University;').fetchall()
    print(results)

[(0, 'University of Illinois at Urbana-Champaign', 'Champaign, IL'), (1, 'Harvard University', 'Cambridge, MA'), (2, 'Harper College', 'Palatine, IL'), (3, 'Northwestern University', 'Evanston, IL'), (4, 'Massachusetts Institute of Technology', 'Cambridge, MA'), (5, 'Yale University', 'New Haven, CT')]


In [61]:
fake.first_name()
fake.last_name()

'Jones'

In [63]:
with pool.connect() as con: 
    con.execute('delete from User;')

In [67]:
with pool.connect() as con: 
    statement = sqlalchemy.text(
        'insert into User (userID, userName, email, universityID) values (:userID, :userName, :email, :universityID);'
    )

    for i in range(3001,5000): 
        first_name = fake.first_name()
        last_name = fake.last_name()
        email = first_name + '.' + last_name + '@example.com'
        con.execute(statement, userID=i, userName=first_name + ' ' + last_name, email=email, universityID=random.randint(0, 5))

In [65]:
# with pool.connect() as con: 
#     con.execute('delete from SubscriptionService where serviceName = "Netflix";')

In [47]:
with pool.connect() as con: 
    statement = sqlalchemy.text(
        'insert into SubscriptionService (serviceName, price, maxMembers) values (:serviceName, :price, :maxMembers);'
    )

    con.execute(statement, serviceName='Netflix', price=19.99, maxMembers=4)
    con.execute(statement, serviceName='Spotify', price=15.99, maxMembers=6)
    con.execute(statement, serviceName='Disney+', price=19.99, maxMembers=4)
    con.execute(statement, serviceName='Amazon Prime Video', price=14.99, maxMembers=3)
    con.execute(statement, serviceName='Hulu', price=12.99, maxMembers=2)

In [66]:
with pool.connect() as con: 
    results = con.execute('select * from SubscriptionService;').fetchall()
    print(results)

[('Amazon Prime Video', Decimal('14.99'), 3), ('Disney+', Decimal('19.99'), 4), ('Hulu', Decimal('12.99'), 2), ('Netflix', Decimal('19.99'), 4), ('Spotify', Decimal('15.99'), 6)]


In [86]:
services = ['Netflix', 'Spotify', 'Disney+', 'Amazon Prime Video', 'Hulu']
n_families = 1000
n_users = 5000
n_members = 3000
userIDs = [i for i in range(n_users)]
familyIDs = [i for i in range(n_families)]
leaderIDs = random.choices(userIDs, k=n_families)#.sample(userIDs, n_families)
accessTypes = ['Open' if random.random() < 0.7 else 'Closed' for _ in range(n_families)]
serviceNames = [services[random.randint(0, 4)] for _ in range(n_families)]

memberIDs = [i for i in range(n_members)]

statuses = ['Accepted', 'Pending', 'Rejected']
member_statuses = random.choices(statuses, weights=[6, 1, 1], k=n_members)

memberships = []
# Do all leader memberships
for i in range(len(leaderIDs)): 
    memberships.append((leaderIDs[i], familyIDs[i], 'Accepted'))

# Do all other memberships
for i in range(n_members): 
    userID = userIDs[random.randint(0, len(userIDs) - 1)]
    familyID = familyIDs[random.randint(0, len(familyIDs) - 1)]
    memberships.append((userID, familyID, member_statuses[i]))

In [87]:
len(set(memberships))

4000

In [98]:
with pool.connect() as con: 
    statement = sqlalchemy.text(
        'insert into Family (familyID, leaderID, accessType, serviceName) values (:familyID, :leaderID, :accessType, :serviceName);'
    )

    for i in range(len(familyIDs)): 
        con.execute(statement, familyID=familyIDs[i], leaderID=leaderIDs[i], accessType=accessTypes[i], serviceName=serviceNames[i])


In [99]:
with pool.connect() as con: 
    statement = sqlalchemy.text(
        'insert into Membership (memberID, familyID, memberStatus) values (:memberID, :familyID, :memberStatus);'
    )

    for membership in memberships: 
        con.execute(statement, memberID=membership[0], familyID=membership[1], memberStatus=membership[2])


In [105]:
with pool.connect() as con: 
    result = con.execute('select count(*) from Family;').fetchall()
    print(result)

[(1000,)]


In [123]:
with pool.connect() as con: 
    statement = '''
    select un.universityName, count(*) as numPending
    from University un
    natural join User us
    join Membership m
    on m.memberID = us.userID
    where m.memberStatus = "Pending"
    group by un.universityName
    order by numPending desc;
    '''
    result = con.execute(statement).fetchall()
    for row in result: 
        print(row)

('Harper College', 73)
('Northwestern University', 61)
('Yale University', 61)
('University of Illinois at Urbana-Champaign', 60)
('Harvard University', 57)
('Massachusetts Institute of Technology', 57)


In [118]:
with pool.connect() as con: 
    statement = '''
    select un.universityName, ss.serviceName, count(*) as numUsers
    from User us
    join Membership m
        on m.memberID = us.userID
    natural join Family f
    natural join SubscriptionService ss
    natural join University un
    where m.memberStatus = "Accepted"
    group by un.universityID, ss.serviceName
    order by un.universityName asc, numUsers desc
    limit 15;
    '''
    result = con.execute(statement).fetchall()
    for row in result: 
        print(row)

('Harper College', 'Netflix', 135)
('Harper College', 'Spotify', 131)
('Harper College', 'Amazon Prime Video', 126)
('Harper College', 'Hulu', 99)
('Harper College', 'Disney+', 90)
('Harvard University', 'Netflix', 107)
('Harvard University', 'Disney+', 106)
('Harvard University', 'Amazon Prime Video', 100)
('Harvard University', 'Spotify', 100)
('Harvard University', 'Hulu', 93)
('Massachusetts Institute of Technology', 'Spotify', 112)
('Massachusetts Institute of Technology', 'Amazon Prime Video', 104)
('Massachusetts Institute of Technology', 'Netflix', 103)
('Massachusetts Institute of Technology', 'Disney+', 93)
('Massachusetts Institute of Technology', 'Hulu', 93)


In [149]:
with pool.connect() as con: 
    statement = '''
    explain analyze
    select un.universityName, count(*) as numPending
    from University un
    natural join User us
    join Membership m
    on m.memberID = us.userID
    where m.memberStatus = "Pending"
    group by un.universityName
    order by numPending desc;
    '''
    result = con.execute(statement).fetchall()
    for row in result[0][0].splitlines():
        print(row)

-> Sort: numPending DESC  (actual time=2.681..2.681 rows=6 loops=1)
    -> Table scan on <temporary>  (actual time=0.001..0.001 rows=6 loops=1)
        -> Aggregate using temporary table  (actual time=2.662..2.663 rows=6 loops=1)
            -> Nested loop inner join  (cost=683.50 rows=400) (actual time=0.079..2.358 rows=369 loops=1)
                -> Nested loop inner join  (cost=543.50 rows=400) (actual time=0.072..2.029 rows=369 loops=1)
                    -> Filter: (m.memberStatus = 'Pending')  (cost=403.50 rows=400) (actual time=0.061..1.475 rows=369 loops=1)
                        -> Table scan on m  (cost=403.50 rows=4000) (actual time=0.054..1.112 rows=4000 loops=1)
                    -> Filter: (us.universityID is not null)  (cost=0.25 rows=1) (actual time=0.001..0.001 rows=1 loops=369)
                        -> Single-row index lookup on us using PRIMARY (userID=m.memberID)  (cost=0.25 rows=1) (actual time=0.001..0.001 rows=1 loops=369)
                -> Single-row ind

In [161]:
with pool.connect() as con: 
    statement = '''
    alter table University
    add index uniName(universityName)
    '''
    con.execute(statement)

OperationalError: (pymysql.err.OperationalError) (1061, "Duplicate key name 'uniName'")
[SQL: 
    alter table University
    add index uniName(universityName)
    ]
(Background on this error at: http://sqlalche.me/e/13/e3q8)

In [159]:
with pool.connect() as con: 
    statement = '''
    explain analyze
    select un.universityName, count(*) as numPending
    from University un
    natural join User us
    join Membership m
    on m.memberID = us.userID
    where m.memberStatus = "Pending"
    group by un.universityName
    order by numPending desc;
    '''
    result = con.execute(statement).fetchall()
    for row in result[0][0].splitlines():
        print(row)

-> Sort: numPending DESC  (actual time=2.988..2.988 rows=6 loops=1)
    -> Table scan on <temporary>  (actual time=0.001..0.002 rows=6 loops=1)
        -> Aggregate using temporary table  (actual time=2.927..2.928 rows=6 loops=1)
            -> Nested loop inner join  (cost=683.50 rows=400) (actual time=0.368..2.633 rows=369 loops=1)
                -> Nested loop inner join  (cost=543.50 rows=400) (actual time=0.362..2.310 rows=369 loops=1)
                    -> Filter: (m.memberStatus = 'Pending')  (cost=403.50 rows=400) (actual time=0.351..1.698 rows=369 loops=1)
                        -> Table scan on m  (cost=403.50 rows=4000) (actual time=0.344..1.362 rows=4000 loops=1)
                    -> Filter: (us.universityID is not null)  (cost=0.25 rows=1) (actual time=0.001..0.001 rows=1 loops=369)
                        -> Single-row index lookup on us using PRIMARY (userID=m.memberID)  (cost=0.25 rows=1) (actual time=0.001..0.001 rows=1 loops=369)
                -> Single-row ind

In [162]:
with pool.connect() as con: 
    statement = '''
    drop index uniName on University;
    '''
    con.execute(statement)

In [164]:
with pool.connect() as con: 
    statement = '''
    alter table Membership
    add index memId(memberID);
    '''
    con.execute(statement)

In [165]:
with pool.connect() as con: 
    statement = '''
    explain analyze
    select un.universityName, count(*) as numPending
    from University un
    natural join User us
    join Membership m
    on m.memberID = us.userID
    where m.memberStatus = "Pending"
    group by un.universityName
    order by numPending desc;
    '''
    result = con.execute(statement).fetchall()
    for row in result[0][0].splitlines():
        print(row)

-> Sort: numPending DESC  (actual time=2.596..2.596 rows=6 loops=1)
    -> Table scan on <temporary>  (actual time=0.001..0.001 rows=6 loops=1)
        -> Aggregate using temporary table  (actual time=2.579..2.580 rows=6 loops=1)
            -> Nested loop inner join  (cost=683.50 rows=400) (actual time=0.084..2.288 rows=369 loops=1)
                -> Nested loop inner join  (cost=543.50 rows=400) (actual time=0.078..1.950 rows=369 loops=1)
                    -> Filter: (m.memberStatus = 'Pending')  (cost=403.50 rows=400) (actual time=0.064..1.374 rows=369 loops=1)
                        -> Table scan on m  (cost=403.50 rows=4000) (actual time=0.056..1.038 rows=4000 loops=1)
                    -> Filter: (us.universityID is not null)  (cost=0.25 rows=1) (actual time=0.001..0.001 rows=1 loops=369)
                        -> Single-row index lookup on us using PRIMARY (userID=m.memberID)  (cost=0.25 rows=1) (actual time=0.001..0.001 rows=1 loops=369)
                -> Single-row ind

In [167]:
with pool.connect() as con: 
    statement = '''
    drop index memId on Membership;
    '''
    # alter table User
    # add index useId(userID);
    con.execute(statement)

In [168]:
with pool.connect() as con: 
    statement = '''
    alter table User
    add index useId(userID);
    '''
    con.execute(statement)

In [169]:
with pool.connect() as con: 
    statement = '''
    explain analyze
    select un.universityName, count(*) as numPending
    from University un
    natural join User us
    join Membership m
    on m.memberID = us.userID
    where m.memberStatus = "Pending"
    group by un.universityName
    order by numPending desc;
    '''
    result = con.execute(statement).fetchall()
    for row in result[0][0].splitlines():
        print(row)

-> Sort: numPending DESC  (actual time=2.956..2.956 rows=6 loops=1)
    -> Table scan on <temporary>  (actual time=0.001..0.002 rows=6 loops=1)
        -> Aggregate using temporary table  (actual time=2.935..2.936 rows=6 loops=1)
            -> Nested loop inner join  (cost=683.50 rows=400) (actual time=0.064..2.585 rows=369 loops=1)
                -> Nested loop inner join  (cost=543.50 rows=400) (actual time=0.058..2.263 rows=369 loops=1)
                    -> Filter: (m.memberStatus = 'Pending')  (cost=403.50 rows=400) (actual time=0.048..1.599 rows=369 loops=1)
                        -> Table scan on m  (cost=403.50 rows=4000) (actual time=0.041..1.216 rows=4000 loops=1)
                    -> Filter: (us.universityID is not null)  (cost=0.25 rows=1) (actual time=0.002..0.002 rows=1 loops=369)
                        -> Single-row index lookup on us using PRIMARY (userID=m.memberID)  (cost=0.25 rows=1) (actual time=0.001..0.001 rows=1 loops=369)
                -> Single-row ind

In [10]:
def run_query(statement):
    with pool.connect() as con: 
        result = con.execute(statement).fetchall()
        if result: 
            for row in result:  
                print(row)

In [ ]:
run_

In [5]:
def run_analyze(statement): 
    with pool.connect() as con: 
        result = con.execute(statement).fetchall()
        for row in result[0][0].splitlines(): 
            print(row)

In [11]:
run_query('''
show index from User;
''')

('User', 0, 'PRIMARY', 1, 'userID', 'A', 4793, None, None, '', 'BTREE', '', '', 'YES', None)
('User', 1, 'universityID', 1, 'universityID', 'A', 6, None, None, 'YES', 'BTREE', '', '', 'YES', None)
('User', 1, 'useId', 1, 'userID', 'A', 4793, None, None, '', 'BTREE', '', '', 'YES', None)
